In [23]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

### 31. How to fill an intermittent time series so all missing dates show up with values of previous non-missing date?

In [65]:
ser = pd.Series([1,10,3,np.nan], index=pd.to_datetime(['2000-01-01', '2000-01-03', '2000-01-06', '2000-01-08']))
ser

2000-01-01     1.0
2000-01-03    10.0
2000-01-06     3.0
2000-01-08     NaN
dtype: float64

In [66]:
ser.resample('D').ffill()

2000-01-01     1.0
2000-01-02     1.0
2000-01-03    10.0
2000-01-04    10.0
2000-01-05    10.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     NaN
Freq: D, dtype: float64

In [59]:
dt = pd.Series(ser.index)
dt

0   2000-01-01
1   2000-01-03
2   2000-01-06
3   2000-01-08
dtype: datetime64[ns]

In [60]:
dt = dt.astype(np.datetime64)
dt = dt.append(dt.dt.date+timedelta(days=1)).reset_index(drop=True)

In [61]:
dt = dt.sort_values()

In [62]:
ser = ser.reindex(dt)
ser

2000-01-01     1.0
2000-01-02     NaN
2000-01-03    10.0
2000-01-04     NaN
2000-01-06     3.0
2000-01-07     NaN
2000-01-08     NaN
2000-01-09     NaN
dtype: float64

In [64]:
ser.ffill()

2000-01-01     1.0
2000-01-02     1.0
2000-01-03    10.0
2000-01-04    10.0
2000-01-06     3.0
2000-01-07     3.0
2000-01-08     3.0
2000-01-09     3.0
dtype: float64

### 32. How to compute the autocorrelations of a numeric series?

In [67]:
ser = pd.Series(np.arange(20) + np.random.normal(1, 10, 20))
ser

0      9.162181
1    -11.533194
2      3.616178
3     -3.147835
4     -4.873950
5      5.018274
6     18.467837
7     10.902159
8     13.173130
9     20.319151
10     4.749274
11    -3.763683
12    22.048271
13    23.771586
14    30.950764
15    23.808406
16    28.750701
17    22.742696
18    15.438100
19    25.033182
dtype: float64

In [84]:
ac = []
for i in range(1,11):
    ac.append(ser.autocorr(i).round(2))

In [85]:
array = np.array(ac)
array

array([0.58, 0.44, 0.41, 0.24, 0.06, 0.4 , 0.7 , 0.37, 0.29, 0.36])

In [86]:
array.argsort()[-1]+1

7

### 33. How to import only every nth row from a csv file to create a dataframe?

In [101]:
n=50
df = pd.read_csv('BostonHousing.csv', skiprows=lambda x: True if x%(n-1)!=0 else False)
df

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.25387,0,6.91,0,0.448,5.399,95.3,5.8700,3,233,17.9,396.90,30.81,14.4
1,0.12083,0,2.89,0,0.445,8.069,76.0,3.4952,2,276,18.0,396.90,4.21,38.7
2,2.15505,0,19.58,0,0.871,5.628,100.0,1.5166,5,403,14.7,169.27,16.65,15.6
3,0.01381,80,0.46,0,0.422,7.875,32.0,5.6484,4,255,14.4,394.23,2.97,50.0
4,0.20608,22,5.86,0,0.431,5.593,76.5,7.9549,7,330,19.1,372.49,12.50,17.6
5,0.08265,0,13.92,0,0.437,6.127,18.4,5.5027,4,289,16.0,396.90,8.58,23.9
6,0.02498,0,1.89,0,0.518,6.540,59.7,6.2669,1,422,15.9,389.96,8.65,16.5
7,5.29305,0,18.10,0,0.700,6.051,82.5,2.1678,24,666,20.2,378.38,18.76,23.2
8,22.05110,0,18.10,0,0.740,5.818,92.4,1.8662,24,666,20.2,391.45,22.11,10.5
9,0.18337,0,27.74,0,0.609,5.414,98.3,1.7554,4,711,20.1,344.05,23.97,7.0


In [111]:
n=50
df = pd.read_csv('BostonHousing.csv', chunksize=50)
df2 = pd.DataFrame()
df2 = pd.concat([chunk.iloc[0,:] for chunk in df], axis=1)
df2.T

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
50,0.08873,21.0,5.64,0.0,0.439,5.963,45.7,6.8147,4.0,243.0,16.8,395.56,13.45,19.7
100,0.14866,0.0,8.56,0.0,0.520,6.727,79.9,2.7778,5.0,384.0,20.9,394.76,9.42,27.5
150,1.65660,0.0,19.58,0.0,0.871,6.122,97.3,1.6180,5.0,403.0,14.7,372.80,14.10,21.5
200,0.01778,95.0,1.47,0.0,0.403,7.135,13.9,7.6534,3.0,402.0,17.0,384.30,4.45,32.9
250,0.14030,22.0,5.86,0.0,0.431,6.487,13.0,7.3967,7.0,330.0,19.1,396.28,5.90,24.4
300,0.04417,70.0,2.24,0.0,0.400,6.871,47.4,7.8278,5.0,358.0,14.8,390.86,6.07,24.8
350,0.06211,40.0,1.25,0.0,0.429,6.490,44.4,8.7921,1.0,335.0,19.7,396.90,5.98,22.9
400,25.04610,0.0,18.10,0.0,0.693,5.987,100.0,1.5888,24.0,666.0,20.2,396.90,26.77,5.6
450,6.71772,0.0,18.10,0.0,0.713,6.749,92.6,2.3236,24.0,666.0,20.2,0.32,17.44,13.4


### 34. How to change column values when importing csv to a dataframe?

In [112]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer: Union[ForwardRef('PathLike[str]'), str, IO[~T], io.RawIOBase, io.BufferedIOBase, io.TextIOBase, _io.TextIOWrapper, mmap.mmap], sep=<object object at 0x0000021EBD6E5510>, delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_

In [115]:
df = pd.read_csv('BostonHousing.csv', converters={'medv':lambda x: 'High' if float(x) > 25 else 'Low'})
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,Low
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,Low
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,High
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,High
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,High


### 35. How to create a dataframe with rows as strides from a given series?

In [126]:
L = pd.Series(range(15))
n = 4
result = []
for i in range(0,int(L.shape[0]/n)+3):
    start = int((i*n) if i==0 else (i*n)/2)
    end = start + n
    result.append(np.array(L[start:end]))
result

[array([0, 1, 2, 3], dtype=int64),
 array([2, 3, 4, 5], dtype=int64),
 array([4, 5, 6, 7], dtype=int64),
 array([6, 7, 8, 9], dtype=int64),
 array([ 8,  9, 10, 11], dtype=int64),
 array([10, 11, 12, 13], dtype=int64)]

In [130]:
L.shape[0]//n

3

### 36. How to import only specified columns from a csv file?

In [132]:
df = pd.read_csv('BostonHousing.csv', usecols=['crim','medv'])
df.head()

,crim,medv
0,0.00632,24.0
1,0.02731,21.6
2,0.02729,34.7
3,0.03237,33.4
4,0.06905,36.2


### 37. How to get the nrows, ncolumns, datatype, summary stats of each column of a dataframe? Also get the array and list equivalent.

In [135]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv')
df.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


In [136]:
df.shape

(93, 27)

In [138]:
df.describe()

,Min.Price,Price,Max.Price,MPG.city,MPG.highway,EngineSize,Horsepower,RPM,Rev.per.mile,Fuel.tank.capacity,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight
count,86.000000,91.000000,88.000000,84.000000,91.000000,91.000000,86.000000,90.000000,87.000000,85.000000,91.000000,89.000000,92.000000,87.000000,88.000000,89.000000,74.000000,86.000000
mean,17.118605,19.616484,21.459091,22.404762,29.065934,2.658242,144.000000,5276.666667,2355.000000,16.683529,5.076923,182.865169,103.956522,69.448276,38.954545,27.853933,13.986486,3104.593023
std,8.828290,9.724280,10.696563,5.841520,5.370293,1.045845,53.455204,605.554811,486.916616,3.375748,1.045953,14.792651,6.856317,3.778023,3.304157,3.018129,3.120824,600.129993
min,6.700000,7.400000,7.900000,15.000000,20.000000,1.000000,55.000000,3800.000000,1320.000000,9.200000,2.000000,141.000000,90.000000,60.000000,32.000000,19.000000,6.000000,1695.000000
25%,10.825000,12.350000,14.575000,18.000000,26.000000,1.800000,100.750000,4800.000000,2017.500000,14.500000,4.000000,174.000000,98.000000,67.000000,36.000000,26.000000,12.000000,2647.500000
50%,14.600000,17.700000,19.150000,21.000000,28.000000,2.300000,140.000000,5200.000000,2360.000000,16.500000,5.000000,181.000000,103.000000,69.000000,39.000000,27.500000,14.000000,3085.000000
75%,20.250000,23.500000,24.825000,25.000000,31.000000,3.250000,170.000000,5787.500000,2565.000000,19.000000,6.000000,192.000000,110.000000,72.000000,42.000000,30.000000,16.000000,3567.500000
max,45.400000,61.900000,80.000000,46.000000,50.000000,5.700000,300.000000,6500.000000,3755.000000,27.000000,8.000000,219.000000,119.000000,78.000000,45.000000,36.000000,22.000000,4105.000000


In [146]:
df.dtypes

Manufacturer           object
Model                  object
Type                   object
Min.Price             float64
Price                 float64
Max.Price             float64
MPG.city              float64
MPG.highway           float64
AirBags                object
DriveTrain             object
Cylinders              object
EngineSize            float64
Horsepower            float64
RPM                   float64
Rev.per.mile          float64
Man.trans.avail        object
Fuel.tank.capacity    float64
Passengers            float64
Length                float64
Wheelbase             float64
Width                 float64
Turn.circle           float64
Rear.seat.room        float64
Luggage.room          float64
Weight                float64
Origin                 object
Make                   object
dtype: object

In [147]:
df.values

array([['Acura', 'Integra', 'Small', ..., 2705.0, 'non-USA',
        'Acura Integra'],
       [nan, 'Legend', 'Midsize', ..., 3560.0, 'non-USA', 'Acura Legend'],
       ['Audi', '90', 'Compact', ..., 3375.0, 'non-USA', 'Audi 90'],
       ...,
       ['Volkswagen', 'Corrado', 'Sporty', ..., 2810.0, 'non-USA',
        'Volkswagen Corrado'],
       ['Volvo', '240', 'Compact', ..., 2985.0, 'non-USA', 'Volvo 240'],
       [nan, '850', 'Midsize', ..., 3245.0, 'non-USA', 'Volvo 850']],
      dtype=object)

In [148]:
df.values.tolist()

[['Acura',
  'Integra',
  'Small',
  12.9,
  15.9,
  18.8,
  25.0,
  31.0,
  'None',
  'Front',
  '4',
  1.8,
  140.0,
  6300.0,
  2890.0,
  'Yes',
  13.2,
  5.0,
  177.0,
  102.0,
  68.0,
  37.0,
  26.5,
  nan,
  2705.0,
  'non-USA',
  'Acura Integra'],
 [nan,
  'Legend',
  'Midsize',
  29.2,
  33.9,
  38.7,
  18.0,
  25.0,
  'Driver & Passenger',
  'Front',
  '6',
  3.2,
  200.0,
  5500.0,
  2335.0,
  'Yes',
  18.0,
  5.0,
  195.0,
  115.0,
  71.0,
  38.0,
  30.0,
  15.0,
  3560.0,
  'non-USA',
  'Acura Legend'],
 ['Audi',
  '90',
  'Compact',
  25.9,
  29.1,
  32.3,
  20.0,
  26.0,
  'Driver only',
  'Front',
  '6',
  2.8,
  172.0,
  5500.0,
  2280.0,
  'Yes',
  16.9,
  5.0,
  180.0,
  102.0,
  67.0,
  37.0,
  28.0,
  14.0,
  3375.0,
  'non-USA',
  'Audi 90'],
 ['Audi',
  '100',
  'Midsize',
  nan,
  37.7,
  44.6,
  19.0,
  26.0,
  'Driver & Passenger',
  nan,
  '6',
  nan,
  172.0,
  5500.0,
  2535.0,
  nan,
  21.1,
  6.0,
  193.0,
  106.0,
  nan,
  37.0,
  31.0,
  17.0,
  3405.0,


### 38. How to extract the row and column number of a particular cell with given criterion?

In [149]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv')
df.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


In [153]:
result = df[df['Price'] == df['Price'].max()]
result.index, list(result.columns).index('Price')

(Int64Index([58], dtype='int64'), 4)

In [154]:
row, col = np.where(df.values == np.max(df.Price))
row, col

(array([58], dtype=int64), array([4], dtype=int64))

In [155]:
df.at[row[0], 'Price']

61.9

### 39. How to rename a specific columns in a dataframe?

In [157]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv')
print(df.columns)

Index(['Manufacturer', 'Model', 'Type', 'Min.Price', 'Price', 'Max.Price',
       'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Man.trans.avail',
       'Fuel.tank.capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn.circle', 'Rear.seat.room', 'Luggage.room', 'Weight', 'Origin',
       'Make'],
      dtype='object')


In [159]:
df = df.rename(%colors{'Type':'CarType'},axis=1)

In [160]:
df.head()

,Manufacturer,Model,CarType,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


In [161]:
df.columns = df.columns.str.replace('.','_')
df.head()

<ipython-input-161-88a1af3e170f>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.','_')


,Manufacturer,Model,CarType,Min_Price,Price,Max_Price,MPG_city,MPG_highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn_circle,Rear_seat_room,Luggage_room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25.0,31.0,None,Front,...,5.0,177.0,102.0,68.0,37.0,26.5,NaN,2705.0,non-USA,Acura Integra
1,NaN,Legend,Midsize,29.2,33.9,38.7,18.0,25.0,Driver & Passenger,Front,...,5.0,195.0,115.0,71.0,38.0,30.0,15.0,3560.0,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20.0,26.0,Driver only,Front,...,5.0,180.0,102.0,67.0,37.0,28.0,14.0,3375.0,non-USA,Audi 90
3,Audi,100,Midsize,NaN,37.7,44.6,19.0,26.0,Driver & Passenger,NaN,...,6.0,193.0,106.0,NaN,37.0,31.0,17.0,3405.0,non-USA,Audi 100
4,BMW,535i,Midsize,NaN,30.0,NaN,22.0,30.0,NaN,Rear,...,4.0,186.0,109.0,69.0,39.0,27.0,13.0,3640.0,non-USA,BMW 535i


### 40. How to check if a dataframe has any missing values?

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/selva86/datasets/master/Cars93_miss.csv')
df.head()